## Library imports

In [70]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
import librosa

from tqdm import tqdm
tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go


In [3]:
from util import data_address
data_address 

'./data/'

In [12]:
FILE_DIR = data_address+"VOiCES_devkit/"

## Read original file

In [4]:
df_raw = pd.read_pickle(data_address+"df_merged.pkl")
df_raw

,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,...,transcript,noisy_length,noisy_sr,noisy_time,source_length,source_sr,source_time,category,frequency,signal
0,0,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,...,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test,16000,"[406, 725, 603, 562, 418, 343, 257, 254, 288, ..."
1,1,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,...,and i got on with the lesson very badly i am s...,255040,16000,15.940,255040,16000,15.940,test,16000,"[285, 611, 451, 492, 377, 313, 201, 19, -114, ..."
2,2,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,...,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test,16000,"[145, 76, -353, -529, -706, -886, -719, -536, ..."
3,3,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,...,and ran upstairs to put away the host of littl...,258000,16000,16.125,258000,16000,16.125,test,16000,"[-147, -332, -326, -291, -211, -85, -2, 66, 10..."
4,4,130550,10,babb,distant-16k/speech/test/rm4/babb/sp1447/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...,rm4,26,...,he was inexhaustibly voluble,58720,16000,3.670,58720,16000,3.670,test,16000,"[94, 179, 164, 154, 115, 78, 76, 84, 52, 28, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,12795,162129,160,none,distant-16k/speech/train/rm4/none/sp2673/Lab41...,M,1,Lab41-SRI-VOiCES-rm4-none-sp2673-ch162129-sg00...,rm4,27,...,and away with it we ought all to say if social...,239840,16000,14.990,239840,16000,14.990,train,16000,"[-64, -133, -146, -149, -122, -118, -133, -174..."
19196,12796,75242,170,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,5,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,rm4,16,...,and zip dropped from the boy's arms not into t...,269040,16000,16.815,269040,16000,16.815,train,16000,"[-372, -677, -647, -637, -622, -614, -662, -68..."
19197,12797,75242,170,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,rm4,16,...,and zip dropped from the boy's arms not into t...,269040,16000,16.815,269040,16000,16.815,train,16000,"[110, 178, 162, 165, 214, 241, 224, 213, 178, ..."
19198,12798,14653,160,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch014653-sg00...,rm4,3,...,for for the sake of my reputation i suggested ...,256080,16000,16.005,256080,16000,16.005,train,16000,"[16, 30, 8, 2, 8, 31, 61, 62, 33, -6, -2, 35, ..."


In [5]:
df_signal = df_raw[['index','category','frequency','signal']]
df_signal

,index,category,frequency,signal
0,0,test,16000,"[406, 725, 603, 562, 418, 343, 257, 254, 288, ..."
1,1,test,16000,"[285, 611, 451, 492, 377, 313, 201, 19, -114, ..."
2,2,test,16000,"[145, 76, -353, -529, -706, -886, -719, -536, ..."
3,3,test,16000,"[-147, -332, -326, -291, -211, -85, -2, 66, 10..."
4,4,test,16000,"[94, 179, 164, 154, 115, 78, 76, 84, 52, 28, -..."
...,...,...,...,...
19195,12795,train,16000,"[-64, -133, -146, -149, -122, -118, -133, -174..."
19196,12796,train,16000,"[-372, -677, -647, -637, -622, -614, -662, -68..."
19197,12797,train,16000,"[110, 178, 162, 165, 214, 241, 224, 213, 178, ..."
19198,12798,train,16000,"[16, 30, 8, 2, 8, 31, 61, 62, 33, -6, -2, 35, ..."


## quick process of signal

In [7]:
type(df_raw['signal'][0])

numpy.ndarray

In [21]:
def apply_simple_process_signal(row):
    row['signal_max'] = row['signal'].max()
    row['signal_min'] = row['signal'].min()
    row['signal_mean'] = row['signal'].mean()
    row['signal_count'] = row['signal'].size
    row['signal_count_positive'] = np.count_nonzero(row['signal']>0)
    row['signal_count_negative'] = np.count_nonzero(row['signal']<0)
    row['signal_count_zero'] = np.count_nonzero(row['signal']==0)
    return row

In [22]:
df_signal_processsed = df_raw.progress_apply(apply_simple_process_signal, axis=1)
df_signal_processsed

100%|██████████| 19200/19200 [00:46<00:00, 415.54it/s]


,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,...,category,frequency,signal,signal_max,signal_min,signal_mean,signal_count,signal_positive,signal_negative,signal_zero
0,0,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,...,test,16000,"[406, 725, 603, 562, 418, 343, 257, 254, 288, ...",14940,-14614,-0.015005,255040,125552,129342,146
1,1,32639,160,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,rm4,29,...,test,16000,"[285, 611, 451, 492, 377, 313, 201, 19, -114, ...",14027,-15073,0.073883,255040,127889,127077,74
2,2,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,...,test,16000,"[145, 76, -353, -529, -706, -886, -719, -536, ...",9857,-10585,-0.028159,258000,128626,129298,76
3,3,32658,70,babb,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,rm4,12,...,test,16000,"[-147, -332, -326, -291, -211, -85, -2, 66, 10...",16185,-13671,-0.027550,258000,124656,133210,134
4,4,130550,10,babb,distant-16k/speech/test/rm4/babb/sp1447/Lab41-...,F,1,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...,rm4,26,...,test,16000,"[94, 179, 164, 154, 115, 78, 76, 84, 52, 28, -...",6889,-7245,-0.034877,58720,29624,29052,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,12795,162129,160,none,distant-16k/speech/train/rm4/none/sp2673/Lab41...,M,1,Lab41-SRI-VOiCES-rm4-none-sp2673-ch162129-sg00...,rm4,27,...,train,16000,"[-64, -133, -146, -149, -122, -118, -133, -174...",7926,-5264,-0.062888,239840,118661,120784,395
19196,12796,75242,170,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,5,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,rm4,16,...,train,16000,"[-372, -677, -647, -637, -622, -614, -662, -68...",10418,-8685,-0.061604,269040,134006,134918,116
19197,12797,75242,170,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,rm4,16,...,train,16000,"[110, 178, 162, 165, 214, 241, 224, 213, 178, ...",11344,-13412,0.041314,269040,135017,133788,235
19198,12798,14653,160,none,distant-16k/speech/train/rm4/none/sp1212/Lab41...,F,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch014653-sg00...,rm4,3,...,train,16000,"[16, 30, 8, 2, 8, 31, 61, 62, 33, -6, -2, 35, ...",6780,-7170,0.023211,256080,127122,128582,376


In [49]:
df_signal_processsed.iloc[0]

index                                                              0
chapter                                                        32639
degrees                                                          160
distractor                                                      babb
filename           distant-16k/speech/test/rm4/babb/sp4899/Lab41-...
gender                                                             F
mic                                                                1
query_name         Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...
room                                                             rm4
segment                                                           29
source             source-16k/test/sp4899/Lab41-SRI-VOiCES-src-sp...
speaker                                                         4899
transcript         and i got on with the lesson very badly i am s...
noisy_length                                                  255040
noisy_sr                          

In [48]:
df_signal_processsed.columns

Index(['index', 'chapter', 'degrees', 'distractor', 'filename', 'gender',
       'mic', 'query_name', 'room', 'segment', 'source', 'speaker',
       'transcript', 'noisy_length', 'noisy_sr', 'noisy_time', 'source_length',
       'source_sr', 'source_time', 'category', 'frequency', 'signal',
       'signal_max', 'signal_min', 'signal_mean', 'signal_count',
       'signal_positive', 'signal_negative', 'signal_zero'],
      dtype='object')

### Change categorical variables to numeric

Define the mapping dictionaries

In [51]:
distractor_mapping = {'none':0,'musi': 1, 'babb': 2,'tele': 3,}
gender_mapping = {'F': 0, 'M': 1}
room_mapping = {'rm1': 0, 'rm2': 1,'rm3': 2,'rm4': 3}

Use map() function to convert categorical columns to numeric

In [54]:
df_signal_processsed['distractor'] = df_signal_processsed['distractor'].map(distractor_mapping)
df_signal_processsed['gender'] = df_signal_processsed['gender'].map(gender_mapping)
df_signal_processsed['room'] = df_signal_processsed['room'].map(room_mapping)

In [56]:
df_signal_processsed


,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,...,category,frequency,signal,signal_max,signal_min,signal_mean,signal_count,signal_positive,signal_negative,signal_zero
0,0,32639,160,2,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,0,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,3,29,...,test,16000,"[406, 725, 603, 562, 418, 343, 257, 254, 288, ...",14940,-14614,-0.015005,255040,125552,129342,146
1,1,32639,160,2,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,0,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032639-sg00...,3,29,...,test,16000,"[285, 611, 451, 492, 377, 313, 201, 19, -114, ...",14027,-15073,0.073883,255040,127889,127077,74
2,2,32658,70,2,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,0,5,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,3,12,...,test,16000,"[145, 76, -353, -529, -706, -886, -719, -536, ...",9857,-10585,-0.028159,258000,128626,129298,76
3,3,32658,70,2,distant-16k/speech/test/rm4/babb/sp4899/Lab41-...,0,1,Lab41-SRI-VOiCES-rm4-babb-sp4899-ch032658-sg00...,3,12,...,test,16000,"[-147, -332, -326, -291, -211, -85, -2, 66, 10...",16185,-13671,-0.027550,258000,124656,133210,134
4,4,130550,10,2,distant-16k/speech/test/rm4/babb/sp1447/Lab41-...,0,1,Lab41-SRI-VOiCES-rm4-babb-sp1447-ch130550-sg00...,3,26,...,test,16000,"[94, 179, 164, 154, 115, 78, 76, 84, 52, 28, -...",6889,-7245,-0.034877,58720,29624,29052,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,12795,162129,160,0,distant-16k/speech/train/rm4/none/sp2673/Lab41...,1,1,Lab41-SRI-VOiCES-rm4-none-sp2673-ch162129-sg00...,3,27,...,train,16000,"[-64, -133, -146, -149, -122, -118, -133, -174...",7926,-5264,-0.062888,239840,118661,120784,395
19196,12796,75242,170,0,distant-16k/speech/train/rm4/none/sp1212/Lab41...,0,5,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,3,16,...,train,16000,"[-372, -677, -647, -637, -622, -614, -662, -68...",10418,-8685,-0.061604,269040,134006,134918,116
19197,12797,75242,170,0,distant-16k/speech/train/rm4/none/sp1212/Lab41...,0,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch075242-sg00...,3,16,...,train,16000,"[110, 178, 162, 165, 214, 241, 224, 213, 178, ...",11344,-13412,0.041314,269040,135017,133788,235
19198,12798,14653,160,0,distant-16k/speech/train/rm4/none/sp1212/Lab41...,0,1,Lab41-SRI-VOiCES-rm4-none-sp1212-ch014653-sg00...,3,3,...,train,16000,"[16, 30, 8, 2, 8, 31, 61, 62, 33, -6, -2, 35, ...",6780,-7170,0.023211,256080,127122,128582,376


In [57]:
df_signal_processsed.drop('signal',axis=1).to_pickle(data_address+"df_processed_simple.pkl")

## MFCC

In [40]:
# https://www.programcreek.com/python/example/105370/python_speech_features.mfcc
# https://www.programcreek.com/python/?code=Atul-Anand-Jha%2FSpeaker-Identification-Python%2FSpeaker-Identification-Python-master%2Ffeatureextraction.py
def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
        """extract 20 dim mfcc features from an audio, performs CMS and combines 
        delta to make it 40 dim feature vector"""    

        mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)    
        mfcc_feature = preprocessing.scale(mfcc_feature)
        delta = calculate_delta(mfcc_feature)
        combined = np.hstack((mfcc_feature,delta)) 
        return combined

In [29]:
df_signal['signal'][0]

array([406, 725, 603, ..., 301, 178,  59], dtype=int16)

In [30]:
df_signal['frequency'][0]

16000